# 1. Convolutional Layer Network

Convolutional Neural Networks, or CNNs, are a class of deep neural networks particularly suited for image recognition and classification tasks. They have revolutionized the field of computer vision by achieving remarkable performance in tasks such as object detection, image segmentation, and facial recognition. In this lecture, we will explore the fundamental concepts behind CNNs and understand how they work.


* Convolutional Layers:

The core building blocks of CNNs are convolutional layers, which apply convolution operations to input images.
Each convolutional layer consists of multiple filters (also known as kernels) that slide over the input image, performing element-wise multiplication and summation to produce feature maps.
These filters capture different patterns and features such as edges, textures, and shapes.

* Pooling Layers:

Pooling layers are used to downsample feature maps, reducing computational complexity and controlling overfitting.
Common pooling operations include max pooling and average pooling, which respectively retain the maximum and average values within defined regions.

* Activation Functions:

Activation functions introduce non-linearity into the network, enabling it to learn complex mappings between inputs and outputs.
Popular activation functions include ReLU (Rectified Linear Unit), sigmoid, and tanh.

* Fully Connected Layers:

Fully connected layers connect every neuron in one layer to every neuron in the next layer, allowing the network to learn high-level representations.
These layers are typically used in the final stages of the network for classification or regression tasks.

* Training Process:
1. Forward Propagation:

        During forward propagation, input images are passed through the network, and computations are performed layer by layer to generate predictions.
2. Loss Calculation:

        The difference between the predicted output and the ground truth labels is quantified using a loss function, such as cross-entropy loss for classification tasks.
3. Backward Propagation (Backpropagation):

        Backpropagation is used to compute the gradients of the loss function with respect to the network parameters. These gradients are then used to update the parameters using optimization algorithms like stochastic gradient descent (SGD) or Adam.
4. Iterative Optimization:

        The training process iterates through multiple epochs, with each epoch consisting of forward and backward propagation steps. Over time, the network learns to minimize the loss function and improve its performance on the training data.

<div style="text-align: center;">
    <img src="Images/image-13.png" alt="Alt text" style="display: block; margin: 0 auto;">
</div>

# 2. Semantic Segmentation:

Semantic segmentation extends the capabilities of CNNs by providing pixel-level understanding of images, enabling the partitioning of an image into semantically meaningful regions and assigning class labels to individual pixels.

* Pixel-Level Classification
        
        Unlike image classification tasks that assign a single label to the entire image, semantic segmentation assigns class labels to each pixel within the image, resulting in a detailed semantic understanding of the scene.

* Encoder-Decoder Architectures
        
        Many semantic segmentation models employ encoder-decoder architectures, where the encoder extracts hierarchical features from the input image, and the decoder generates pixel-wise predictions by upsampling the feature maps to the original resolution.

* Loss Functions
        
        Semantic segmentation models are trained using loss functions that measure the discrepancy between predicted pixel-wise labels and ground truth annotations. Common loss functions include cross-entropy loss, dice loss, and intersection-over-union (IoU) loss.


# 2.1. Model Implementation

For the implementation of our semantic segmentation model, it is advisable to employ the following architecture: (Of course, you can create your own architecture)

* TASK: select the suitbale optimizer, loss function and respective metrics

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda

################################################################
def simple_unet_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS):
#Build the model
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    #s = Lambda(lambda x: x / 255)(inputs)   #No need for this if we normalize our inputs beforehand
    s = inputs

    #Contraction path
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
     
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
     
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
     
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    
    #Expansive path 
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
     
    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
     
    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
     
    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
     
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)
     
    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    
    return model

# 2.2.1. Data Preparation for The Training Step

Use the images that you previously labeled.
* TASK:
    * Implement a Python script to:
        * Read each image and mask using OpenCV.
        * Convert them to grayscale.
        * Resize to SIZE x SIZE.
        * Convert to numpy arrays.
        * Append to image_dataset and mask_dataset list.

In [ ]:
SIZE = 512
image_dataset = []  #Many ways to handle data, you can use pandas. Here, we are using a list format.  
mask_dataset = []  #Place holders to define add labels. We will add 0 to all parasitized images and 1 to uninfected.
validation_dataset = []

images = os.listdir(image_directory)
for i, image_name in enumerate(images):    #Remember enumerate method adds a counter and returns the enumerate object
    if (image_name.split('.')[1] == 'tif'):
        #print(image_directory+image_name)
        image = cv2.imread(image_directory+image_name, 0)
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        image_dataset.append(np.array(image))

#Iterate through all images in Uninfected folder, resize to 64 x 64
#Then save into the same numpy array 'dataset' but with label 1

masks = os.listdir(mask_directory)
for i, image_name in enumerate(masks):
    if (image_name.split('.')[1] == 'tif'):
        image = cv2.imread(mask_directory+image_name, 0)
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        mask_dataset.append(np.array(image))

# 2.2.2. Data Preparation for The Training Step (Normalize image pixels and rescale mask pixels for semantic segmentation)

* TASK:
    * Normalize pixel values of images in image_dataset using the normalize function along the channel axis.
    * Ensure pixel values of images are normalized across all channels.
    * Rescale pixel values of masks in mask_dataset to [0, 1] by dividing each pixel value by 255.
    * Add an extra dimension to both datasets to match the model's input shape.

In [ ]:
#Normalize images
image_dataset = np.expand_dims(normalize(np.array(image_dataset), axis=1),3)
#D not normalize masks, just rescale to 0 to 1.
mask_dataset = np.expand_dims((np.array(mask_dataset)),3) /255.

# 2.2.3. Data Preparation for The Training Step (Split image and mask data with train_test_split)

* TASK:
    * Import the train_test_split function from sklearn.model_selection.
    * Split image_dataset and mask_dataset into training and testing sets:
    * Assign 90% of the data to the training set (X_train, y_train).
    * Assign 10% of the data to the testing set (X_test, y_test).
    * Use a test_size of 0.10 to specify the proportion of data for testing.
    * Set random_state to 0 for reproducibility.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(image_dataset, mask_dataset, test_size = 0.10, random_state = 0)

# 2.3. Train The Model

* TASK:
    * Define the dimensions of the image dataset (image_dataset) as follows:
        * IMG_HEIGHT = image_dataset.shape[1]
        * IMG_WIDTH = image_dataset.shape[2]
        * IMG_CHANNELS = image_dataset.shape[3]
    * Implement a function to retrieve a U-Net model with the specified dimensions
    * Train the model using the training data (X_train, y_train) and validate it using the testing data (X_test, y_test)
    * Save the trained model to the specified location

check the images

In [ ]:
#Sanity check, view few mages
import random
import numpy as np
image_number = random.randint(0, len(X_train))
plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.imshow(np.reshape(X_train[image_number], (512, 512)), cmap='gray')
plt.subplot(122)
plt.imshow(np.reshape(y_train[image_number], (512, 512)), cmap='gray')
plt.show()


In [ ]:
IMG_HEIGHT = image_dataset.shape[1]
IMG_WIDTH  = image_dataset.shape[2]
IMG_CHANNELS = image_dataset.shape[3]

In [ ]:
def get_model():
    return simple_unet_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

In [ ]:
model = get_model()

In [ ]:
#If starting with pre-trained weights. 
#model.load_weights('mitochondria_gpu_tf1.4.hdf5')

history = model.fit(X_train, y_train, 
                    batch_size = 16, 
                    verbose=1, 
                    epochs=5, 
                    validation_data=(X_test, y_test), 
                    shuffle=False)

model.save(r'D:\02-Project\01-Wear Detection\07-Scripts\debug_environment\debug_environment\Model\TEST_Ehsan_Aug_Test.hdf5')

# 2.4. Extracting Accuracy Parameters
Now, let's delve into the process of extracting accuracy parameters from our model. This task involves two main components:

TASK:

* Extracting Accuracy and Loss Diagrams with Epochs:
    * One fundamental aspect of assessing a model's performance is tracking its accuracy and loss over epochs. By visualizing these metrics over the course of training, we gain insights into how well our model is learning and whether it's overfitting or underfitting. We'll extract these diagrams to analyze the trends and make informed decisions about model adjustments.

* Calculating the Intersection over Union (IOU) Level:
    * Another critical metric for evaluating the performance of models, especially in tasks like object detection and semantic segmentation, is the Intersection over Union (IOU). This metric quantifies the overlap between predicted and ground-truth bounding boxes or segmentation masks. By calculating the IOU level, we can gauge how accurately our model is delineating objects or regions of interest within the data.

In [ ]:
#Evaluate the model


	# evaluate model
_, acc = model.evaluate(X_test, y_test)
print("Accuracy = ", (acc * 100.0), "%")


#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history.history['accuracy']
#acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
#val_acc = history.history['val_accuracy']

plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
#IOU
y_pred=model.predict(X_test)
y_pred_thresholded = y_pred > 0.5

intersection = np.logical_and(y_test, y_pred_thresholded)
union = np.logical_or(y_test, y_pred_thresholded)
iou_score = np.sum(intersection) / np.sum(union)
print("IoU socre is: ", iou_score)